In [ ]:
import pandas as pd
import numpy as np
import json
import os
import requests
from datetime import datetime
import time
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib qt
from SEIR import corona_seir_model_population
import math

In [ ]:
API_KEY = 'VJga0fAn'
blacklist = []
retry_set = set()
STATIONS = pd.read_csv('data/stations.csv')

In [ ]:
STATIONS

## Methods

In [ ]:
def drop_null_vals(df,axis='both',subset=[]):
    '''
    Drops columns with all
    nan values from a given 
    data frame.
    
    Parameters
    ----------
    df : DataFrame
        DataFrame for which
        columns are to be
        dropped.
        
    axis : str
        Drops all rows with
        nan if axis=rows,
        all columns if axis=columns,
        and both if axis=both.
        
    subset : list of str
        For all columns in
        subset, remove the
        NaN rows.
    '''
    assert(isinstance(df,pd.DataFrame))
    assert(isinstance(axis,str))
    assert(isinstance(subset,list))
    assert(isinstance(col,str) for col in subset)
    
    df = df.dropna(subset=subset)
    
    if(axis=='rows'):
        df = df.dropna(how='all',axis=0)
    elif(axis=='columns'):
        df = df.dropna(how='all',axis=1)
    elif(axis=='both'):
        df = df.dropna(how='all',axis=0).dropna(how='all',axis=1)
    
    return df

def getWeatherData(latitude,longitude):
    '''
    Returns temperature 
    and humidity data 
    as per the latitude 
    and longitude entered.
    
    Parameters
    ----------
    latitude : float
        Latitude of region
        for fetching the
        weather data
        
    longitude : float
        Longitude of region
        for fetching weather
        data
    '''
    assert(isinstance(latitude,float))
    assert(isinstance(longitude,float))
    
    station = getNearbyStation(latitude,longitude)
    if(station is None):
        return
    else:
        print("Station found for (%f,%f):\n%s"%(latitude,longitude,station))
    url = createHourlyURL(station,START_DATE,END_DATE)
    weather_data = dict()
    print("URL for getting weather data for (%f,%f):\n%s"%(latitude,longitude,url))
    response = requests.get(url)
    code = response.status_code
    print("Got response, status = %f"%(code))
    try:
        df=pd.DataFrame(response.json()['data'])
        date_list = list(df['time_local'].unique())
#         print(retry_set)
        retry_set.discard((latitude,longitude))
        for date in date_list:
            mean_temp=df[df['time_local']==date]['temperature'].mean()
            mean_humidity=df[df['time_local']==date]['humidity'].mean()
            df_dict = dict()
            df_dict['temperature'] = mean_temp
            df_dict['humidity'] = mean_humidity
            weather_data[date] = df_dict
    except:
        print('No data found for (%f,%f)'%(latitude,longitude))
        print("Retry has to be done for (%f,%f)"%(latitude,longitude))
        coordinates = (latitude,longitude)
        retry_set.add(coordinates)
    
    return weather_data

def getNearbyStation(latitude,longitude):
    '''
    Given the latitude and
    longitude of a area,
    returns the nearest station
    to it.
    
    Parameters
    ----------
    latitude : float
        Latitude of region
        for fetching the
        weather data
        
    longitude : float
        Longitude of region
        for fetching weather
        data
    '''
    assert(isinstance(latitude,float))
    assert(isinstance(longitude,float))

    url = createStationURL(latitude,longitude)

    try:
        filter1 = stations['Latitude']==latitude
        filter2 = stations['Longitude']==longitude
        return stations.where(filter1 & filter2).dropna(how='all')['Weather Station ID'][0]
    except:
        try:
            print('Station does not exist in "Stations.csv", fetching from api...')
            response = requests.get(url)
            code = response.status_code
            result = response.json()['data'][0]['id']
            df = pd.DataFrame({"Weather Station ID":[result], "Latitude":[latitude], "Longitude":[longitude]})
            STATIONS.append(df, ignore_index=True)
            print('Station fetched for (%f,%f) is: %s'%(latitude,longitude,result))
            return result
        except Exception as ex:
            print('No station found for (%f,%f). URL: %s'%(latitude,longitude,url))
            print('exception: ',ex)
            if(code == 403):
                print("Retry has to be done for (%f,%f)"%(latitude,longitude))
                retry_set.add((latitude,longitude))
            return
        
def createStationURL(latitude,longitude):
    '''
    Returns station URL
    for given latitude
    and longitude.
    
    Parameters
    ----------
    latitude : float
        Latitude of region
        for fetching the
        weather data
        
    longitude : float
        Longitude of region
        for fetching weather
        data
    '''
    assert(isinstance(latitude,float))
    assert(isinstance(longitude,float))
    
    return 'https://api.meteostat.net/v1/stations/nearby?lat='+str(latitude)+'&lon='+str(longitude)+'&limit=1&key='+API_KEY

def createHourlyURL(station_id,start_date,end_date):
    '''
    Creates weather URL
    for given station,
    start date and end
    date.
    
    Parameters
    ----------
    station_id : str
        Station id of the
        region for which
        data is to be fetched
        
    start_date : str
        Date starting from which
        data is to be fetched.
        
    end_date : str
        Date ending at which
        data is to be fetched
    '''
    assert(isinstance(station_id,str))
    assert(isinstance(start_date,str))
    assert(isinstance(end_date,str))
    
    url = 'https://api.meteostat.net/v1/history/hourly?station='+station_id+'&start='+start_date+'&end='+end_date+'&time_zone=Europe/London&time_format=Y-m-d&key='+API_KEY

    return url

def getCompleteWeatherData(coordinate_list,complete_weather_data=dict(),flag=True):
    '''
    Returns consolidated weather
    data for all the coordinates
    in list of coordinates.
    
    Parameters
    ----------
    coordinate_list : list(tuple)
        List of coordinates to
        be evaluated.
        
    complete_weather_data : dict
        Map of coordinate to
        weather data.
    
    flag : bool
        If True, means some
       coordinate_listts failed, retry
        the failed requests.
    '''
    assert(isinstance(flag,bool))
    assert(isinstance(complete_weather_data,dict))
    assert(isinstance(coordinate_list,list))
    
    try:
        if(flag==True):
            flag=False
            for lat_long in lat_long_list:
                coordinate = (str(lat_long[0]),str(lat_long[1]))
                if((coordinate not in complete_weather_data) and (coordinate not in blacklist)):
                    flag = True
                    complete_weather_data[(str(lat_long[0]),str(lat_long[1]))] = getWeatherData(lat_long[0],lat_long[1])
                    time.sleep(10)
            return getCompleteWeatherData(coordinate_list,complete_weather_data,flag)
        else:
            return complete_weather_data
    except Exception as ex:
        print(ex)
        return complete_weather_data

def getIndexByRegion(province,country,df):
    '''
    Gets index of a region
    from a DataFrame.
    
    Parameters
    ----------
    province : str
        Province for which
        index is to be fetched
        
    Country : str
        Country for which
        index is to be fetched
        
    df : Pandas DataFrame
        DataFrame from which
        index is to be fetched
    '''
    assert(isinstance(country,str))
    assert(isinstance(df,pd.DataFrame))
    
    if(type(province)!=str and np.isnan(province)):
        idx = df[(df['Province/State'].isnull()) & (df['Country/Region']==country)].index
        return idx.to_list()[0]
    else:
        idx = df[(df['Province/State']==province) & (df['Country/Region']==country)].index
        return idx.to_list()[0]

def fetch_province_country_by_region(region):
    '''
    Given a region as
    Province,Country,
    returns the province
    and country or just
    the country if no province
    of the region.
    
    Parameters
    ----------
    region : str
        Region to be parsed
    '''
    assert(isinstance(region,str))
    
    result = region.split(",")
    if(len(result)==2):
        return result[0],result[1].strip()
    else:
        return np.NaN,region
    
def getPopulationByRegion(province,country,df):
    '''
    Given the province and
    country of a region,
    fetches the population
    from the dataframe.
    
    Parameters
    ----------
    province : str
        Province for which
        population is to be fetched
        
    Country : str
        Country for which
        population is to be fetched
        
    df : Pandas DataFrame
        DataFrame from which
        population is to be fetched
    
    '''
    assert(isinstance(country,str))
    assert(isinstance(df,pd.DataFrame))
    
    if(type(province)!=str and np.isnan(province)):
        population = df[(df['Province/State'].isnull()) & (df['Country/Region']==country)].Population
        return population.to_list()[0]
    else:
        population = df[(df['Province/State']==province) & (df['Country/Region']==country)].Population
        return population.to_list()[0]
    
def getDatesByRegion(region,df):
    '''
    Given the province and
    country of a region,
    fetches the infection
    duration from the dataframe.
    
    Parameters
    ----------
    province : str
        Province for which
        dates are to be fetched
        
    Country : str
        Country for which
        dates are to be fetched
        
    df : Pandas DataFrame
        DataFrame from which
        dates are to be fetched
    '''
    assert(isinstance(region,str))
    assert(isinstance(df,pd.DataFrame))
    
    region_row = df[df['Region']==region]
    start_date = region_row['Time series start'].to_list()[0]
    stop_date = region_row['Time series end'].to_list()[0]
    
    return start_date,stop_date

def getLearningRate(region,df):
    '''
    Given the province and
    country of a region,
    fetches the learning
    rate from the dataframe.
    
    Parameters
    ----------
    province : str
        Province for which
        learning rate is
        to be fetched
        
    Country : str
        Country for which
        learning rate is
        to be fetched
        
    df : Pandas DataFrame
        DataFrame from which
        learning rate is
        to be fetched
    '''
    assert(isinstance(region,str))
    assert(isinstance(df,pd.DataFrame))
    
    region_row = df[df['Region']==region]
    learning_rate = region_row['lr'].to_list()
    print("Learning rate:",learning_rate)
    if(math.isnan(learning_rate[0])):
        print("learning rate is null. returning 0.0004")
        return 0.0004
    else:
        return learning_rate[0]

## Data

### 1. covid_19_data.csv

Sno - Serial number<br/>
ObservationDate - Date of the observation in MM/DD/YYYY<br/>
Province/State - Province or state of the observation (Could be empty when missing)<br/>
Country/Region - Country of observation<br/>
Last Update - Time in UTC at which the row is updated for the given province or country. (Not standardised and so please clean before using it)<br/>
Confirmed - Cumulative number of confirmed cases till that date<br/>
Deaths - Cumulative number of of deaths till that date<br/>
Recovered - Cumulative number of recovered cases till that date

### 2. COVID_open_line_list_data.csv and COVID19_line_list_data.csv

Individual level data information
<br/><br/>

ID<br/>
age<br/>
sex<br/>
city<br/>
province<br/>
country<br/>
wuhan(0)_not_wuhan(1)<br/>
latitude<br/>
longitude<br/>
geo_resolution<br/>
date_onset_symptoms<br/>
date_admission_hospital<br/>
date_confirmation<br/>
symptoms<br/>
lives_in_Wuhan<br/>
travel_history_dates<br/>
travel_history_location<br/>
reported_market_exposure<br/>
additional_information<br/>
chronic_disease_binary<br/>
chronic_disease<br/>
source<br/>
sequence_available<br/>
outcome<br/>
date_death_or_discharge<br/>
notes_for_discussion<br/>
location<br/>
admin3<br/>
admin2<br/>
admin1<br/>
country_new<br/>
admin_id<br/>
data_moderator_initials<br/>

### Region wise data

In [ ]:
region_wise_data = drop_null_vals(pd.read_csv('data/covid_19_data.csv'),axis="both")
region_wise_data

### Individual information

In [ ]:
open_line_list = drop_null_vals(pd.read_csv('data/COVID19_open_line_list.csv'),axis='both')
open_line_list

In [ ]:
line_list = drop_null_vals(pd.read_csv('data/COVID19_line_list_data.csv'),'both')
line_list

### Time series data (John Hopkins)

In [ ]:
try:
    confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
#     confirmed_url = 'https://raw.githubusercontent.com/y1singh/Coronavirus-Weather-Modelling/master/data/time_series_covid_19_confirmed.csv'
    deaths_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
#     deaths_url='https://raw.githubusercontent.com/y1singh/Coronavirus-Weather-Modelling/master/data/time_series_covid_19_deaths.csv'
    recovered_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
#     recovered_url = 'https://raw.githubusercontent.com/y1singh/Coronavirus-Weather-Modelling/master/data/time_series_covid_19_recovered.csv'
    
    print('Fetching confirmed data from git...')
    time_series_confirmed = drop_null_vals(pd.read_csv(confirmed_url,error_bad_lines=False))
    print('Fetched confirmed data from git')
    
    time_series_confirmed.to_csv('data/time_series_covid_19_confirmed.csv')
    
    time_series_deaths = drop_null_vals(pd.read_csv(deaths_url,error_bad_lines=False))
    time_series_deaths.to_csv('data/time_series_covid_19_deaths.csv')
    print('Fetched deaths data from git')
    
    time_series_recovered = drop_null_vals(pd.read_csv(recovered_url,error_bad_lines=False))
    time_series_recovered.to_csv('data/time_series_covid_19_recovered.csv')
    print('Fetched recovered data from git')
    
except:
    # data not able to be fetched from git, fetching from local system
    print("Data not able to fetch from git. Using local filesystem.")
    time_series_confirmed = drop_null_vals(pd.read_csv('data/time_series_covid_19_confirmed.csv'))
    time_series_deaths = drop_null_vals(pd.read_csv('data/time_series_covid_19_deaths.csv'))
    time_series_recovered = drop_null_vals(pd.read_csv('data/time_series_covid_19_recovered.csv'))

In [ ]:
population_df = pd.read_csv("data/population_data.csv")

In [ ]:
time_series_confirmed

In [ ]:
# Dropping today's data
time_series_confirmed.drop(time_series_confirmed.columns[len(time_series_confirmed.columns)-1], axis=1, inplace=True)
time_series_deaths.drop(time_series_deaths.columns[len(time_series_deaths.columns)-1], axis=1, inplace=True)
time_series_recovered.drop(time_series_recovered.columns[len(time_series_recovered.columns)-1], axis=1, inplace=True)

In [ ]:
series_confirmed_coord = drop_null_vals(time_series_confirmed,subset=['Lat','Long'])
series_confirmed_coord

### Temperature and humidity data

In [ ]:
lat_long_list = list(series_confirmed_coord[['Lat','Long']].values)

<b><u>API Sample Calls</u></b>

API to fetch station: https://api.meteostat.net/v1/stations/nearby?lat=1.283&lon=103.83&limit=1&key=XXXXXXX<br/>
API to fetch daily historical data(not used any more): https://api.meteostat.net/v1/history/daily?station=10637&start=2017-01-01&end=2017-12-31&key=XXXXXXXX<br/>
API to fetch historical hourly data: https://api.meteostat.net/v1/history/hourly?station=03772&start=2019-05-02&end=2019-05-11&time_zone=Europe/London&time_format=Y-m-d%20H:i&key=XXXXXXXX


In [ ]:
START_DATE = str(datetime.strptime(series_confirmed_coord.columns[4], '%m/%d/%y').date())
END_DATE = str(datetime.strptime(series_confirmed_coord.columns[-1], '%m/%d/%y').date())
complete_weather_data = dict()

In [ ]:
for lat_long in lat_long_list:
    if(str(lat_long[0])+str(lat_long[1]) not in complete_weather_data):
        complete_weather_data[str(lat_long[0])+str(lat_long[1])] = getWeatherData(lat_long[0],lat_long[1])
        time.sleep(60)

complete_weather_data

In [ ]:
temp = getCompleteWeatherData(lat_long_list)
STATIONS.to_csv(r'data\stations.csv', index=False)

In [ ]:
new_dict = {}
for key in list(temp.keys()):
    new_dict[str(key[0])+","+str(key[1])] = temp[key]
    
my_dict = pd.DataFrame(new_dict).dropna(how="all",axis=1).to_dict()
with open('data/weather_mapping_data.json', 'w') as json_file:
    json.dump(my_dict, json_file)

In [ ]:
last_column = time_series_confirmed.columns[len(time_series_confirmed.columns)-1]
sorted_series=time_series_confirmed.sort_values(by=last_column,ascending=False)
sorted_series = sorted_series[sorted_series[last_column]>1000]

In [ ]:
sorted_series.where(sorted_series['Country/Region'] == country,inplace=True)

## Training

In [ ]:
def fit_to_data_population(model,gt_infected,population,region,count,df):
	T_max = gt_infected.shape[0]-1
	max_infected = np.max(gt_infected)
	dt = 1
	T = np.linspace(0,T_max,int(T_max/dt)+1).astype(int)
	N = population
	init_exposed = int(gt_infected[0]*2)
	init_vals = (N-init_exposed-gt_infected[0]),init_exposed,gt_infected[0],0
	gamma2 = 0.03
	rho = 1.0
	total_epochs = 20000
	# lr = 0.04
	# lrd = 0.01
# 	lr = 0.0004/max_infected
	lr = getLearningRate(region,df)/max_infected
	print("learning rate: ",lr)
	lrd = 0.000
	curr_params = 0.2,0.5,0.0,gamma2
	loss_arr = []
	alpha_arr = []
	beta_arr = []
	gamma1_arr = []
	gamma2_arr = []
	for epoch in tqdm(range(total_epochs)):
		curr_lr = lr/(1+epoch*lrd)
		loss_jacobian = epoch_fit_params_corona_seir_population(init_vals,curr_params,T,gt_infected,lr=curr_lr)
		loss_epoch = np.sum(loss_jacobian[0])
		new_alpha = max(0,curr_params[0]+np.sum(loss_jacobian[1]))
		new_beta = max(0,curr_params[1]+np.sum(loss_jacobian[2]))
		new_gamma1 = max(0,curr_params[2]+np.sum(loss_jacobian[3]))
		new_gamma2 = max(0,curr_params[3]+np.sum(loss_jacobian[4]))
		curr_params = new_alpha,new_beta,new_gamma1,new_gamma2
		loss_arr.append(loss_epoch)
		alpha_arr.append(new_alpha)
		beta_arr.append(new_beta)
		gamma1_arr.append(new_gamma1)
		gamma2_arr.append(new_gamma2)
	best_epoch = np.argmin(np.array(loss_arr))
	print("Best learned params: {} {} {}".format(alpha_arr[best_epoch],beta_arr[best_epoch],gamma1_arr[best_epoch]))
	plt.figure(count)
	plt.subplot(221)
	plt.axvline(x=best_epoch,color='k',linestyle='--')
	plt.plot(list(range(total_epochs)),loss_arr)
	plt.ylabel('Total MSE loss')
	plt.xlabel('Epochs')
	plt.subplot(222)
	plt.axvline(x=best_epoch,color='k',linestyle='--')
	plt.plot(list(range(total_epochs)),alpha_arr,label='alpha')
	plt.plot(list(range(total_epochs)),beta_arr,label='beta')
	plt.plot(list(range(total_epochs)),gamma1_arr,label='gamma1')
	# plt.plot(list(range(total_epochs)),gamma2_arr,label='gamma2')
	plt.title('Learning trajectory for '+region)
	plt.ylabel('Parameter value')
	plt.xlabel('Epochs')
	plt.legend()
	# print(init_vals)
	best_params = alpha_arr[best_epoch],beta_arr[best_epoch],gamma1_arr[best_epoch],gamma2
	T_pred = np.linspace(0,10+T_max,int((10+T_max)/dt)+1).astype(int)
	learned_results = model(init_vals,best_params,T_pred)
	# plt.figure(2)
	plt.subplot(212)
	# p = plt.plot(T,sim_results[0],label='GT Susceptible')
	# p = plt.plot(T_pred,learned_results[0]/N,linestyle='--',label='Predicted Susceptible')
	# p = plt.plot(T,sim_results[1],label='GT Exposed')
	# p = plt.plot(T_pred,learned_results[1]/N,linestyle='--',label='Predicted Exposed')
	p = plt.plot(gt_infected[T]/N,label='GT Infected')
	plt.plot(T_pred,learned_results[2]/N,color=p[0].get_color(),linestyle='--',label='Predicted Infected')
	print("error percentage: ",abs(100*(learned_results[2][len(gt_infected)-1]-gt_infected[-1])/gt_infected[-1]))
	# p = plt.plot(T,sim_results[3],label='Recovered')
	p = plt.plot(T_pred,learned_results[3]/N,linestyle='--',label='Predicted Recovered')
	plt.legend()
	plt.ylabel('Fraction of population')
	plt.xlabel('Time (days)')
	plt.title('GT and learned models')
	plt.show()
    
def epoch_fit_params_corona_seir_population(init_vals, init_params, T, infected, lr=1e-2):
	max_infected = np.max(infected)
	S0,E0,I0,R0 = init_vals
	N = S0+E0+I0+R0
	S, E, I , R = [S0], [E0], [I0], [R0]
	loss = [0]
	alpha, beta, gamma1, gamma2 = init_params
	dt = T[1]-T[0]
	jacobian_mat = np.zeros((3,4))
	# updated_alpha, updated_beta, updated_gamma1, updated_gamma2 = [alpha],[beta],[gamma1],[gamma2]
	update_alpha, update_beta, update_gamma1, update_gamma2 = [0],[0],[0],[0]
	# print(infected)
	for idx,t in enumerate(T[1:-3]):
		# print("{} {} {} {} {:.3f} {:.3f} {:.3f} {:3.3f}".format(t,alpha,beta,gamma1,S[-1],E[-1],I[-1],np.max(jacobian_mat)))
		update_mat = np.array([[(1-beta*E[-1]/N),(-beta*S[-1]/N),0],[(beta*E[-1]/N),(1+beta*S[-1]/N-alpha-gamma1),0],[0,alpha,(1-gamma2)]])
		add_mat = np.array([[0,(-S[-1]*(E[-1]/N)),0,0],[(-E[-1]),S[-1]*(E[-1]/N),(-E[-1]),0],[E[-1],0,0,(-I[-1])]])/max_infected
		jacobian_mat = np.matmul(update_mat,jacobian_mat)+add_mat
		min_val = N
		S1 = S[-1] - (beta*S[-1]*E[-1]/N)*dt
		if S1<min_val: min_val=S1
		E1 = E[-1] + (beta*S[-1]*E[-1]/N - alpha*E[-1] - gamma1*E[-1])*dt
		if E1<min_val: min_val=E1
		I1 = I[-1] + (alpha*E[-1] - gamma2*I[-1])*dt
		if I1<min_val: min_val=I1
		R1 = R[-1] + (gamma1*E[-1] + gamma2*I[-1])*dt
		if R1<min_val: min_val=R1
		N1 = S1+E1+I1+R1-4*min_val
		S1 = N*(S1-min_val)/N1
		E1 = N*(E1-min_val)/N1
		I1 = N*(I1-min_val)/N1
		R1 = N*(R1-min_val)/N1
		S.append(S1)
		E.append(E1)
		I.append(I1)
		R.append(R1)
		# print(T[idx+1],infected[T[idx+1]])
		loss.append(((infected[T[idx+1]]-I1)**2)**0.5)
		alpha_update = lr*(infected[T[idx+1]]-I1)*jacobian_mat[2,0]
		beta_update = lr*(infected[T[idx+1]]-I1)*jacobian_mat[2,1]
		gamma1_update = 0 #lr*(infected[T[idx+1]]-I1)*jacobian_mat[2,2]
		gamma2_update = 0 #lr*(infected[idx+1]-I1)*jacobian_mat[2,3]
		alpha_new = alpha+alpha_update
		beta_new = beta+beta_update
		gamma1_new = gamma1+gamma1_update
		gamma2_new = gamma2+gamma2_update
		update_alpha.append(alpha_update)
		update_beta.append(beta_update)
		update_gamma1.append(gamma1_update)
		update_gamma2.append(gamma2_update)
	# return np.stack([loss,alpha_vals_S,alpha_vals_E,alpha_vals_I])
	return np.stack([loss,update_alpha,update_beta,update_gamma1,update_gamma2])

In [ ]:
model_param_df = pd.read_csv('data/model_param_results.csv')

In [ ]:
null_model_param_df = model_param_df[model_param_df['alpha'].isna()]

In [ ]:
regions = null_model_param_df['Region'].to_list()

In [ ]:
province,country = fetch_province_country_by_region(regions[6])

In [ ]:
regions

In [ ]:
count = 0
for region in regions:
    try:
        province,country =fetch_province_country_by_region(region)
        print(province,country)
        region_idx = getIndexByRegion(province,country,time_series_confirmed)
        print(region_idx)
        region_population = int(getPopulationByRegion(province,country,population_df))
        print(region_population)
        time_series_start,time_series_end = getDatesByRegion(region,model_param_df)
        print(getDatesByRegion(region,model_param_df))
        data = time_series_confirmed
        gt_infected = np.array(data.iloc[region_idx,time_series_start:time_series_end]).astype(int)
        plt.figure(count)
#         plt.plot(gt_infected); plt.show()
        fit_to_data_population(corona_seir_model_population,gt_infected,region_population,region,count,model_param_df)
        count+=1
    except:
        print("No data found for %s,%s"%(province,country))